### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 15, 2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
The analysis in this repository is only concerned with the CBECI
`BEST GUESS`. To avoid polution of the .csv we drop all the 
other columns. 
"""
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], 
        axis= 1
    )

In [3]:
# Create the base dataframe on which to merge others by looking at the $0.01 data
df = pd.read_csv(
    "https://ccaf.io/cbeci/api/v1.2.0/download/data?price=0.01", 
    skiprows=1
)

# Drop the unwanted columns
df = drop_standard_columns(df)

# Rename the columns
df.rename(
    columns={
        'Date and Time': 'date', 
        'annualised consumption GUESS, TWh': 0.01
    }, 
    inplace=True
)

In [4]:
# Download the data for all prices and then merge them into one dataframe
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    
    tmp_df = pd.read_csv(
        f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", 
        skiprows=1
    )
    
    tmp_df = drop_standard_columns(tmp_df)
    
    tmp_df.rename(
        columns={
            'Date and Time': 'date', 
            'annualised consumption GUESS, TWh': elec_str
        }, 
        inplace=True
    )

    # Merge the data into the main data frame
    df = df.merge(
        tmp_df, 
        how="inner", 
        on="date"
    )

In [5]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4549,2022-12-31T00:00:00,148.586208,132.211603,111.116038,101.914458,90.485172,82.862069,72.895885,68.903522,61.024572,53.952259,52.481132,43.994596,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4550,2023-01-01T00:00:00,147.979750,134.047846,112.302037,102.434037,91.374154,84.012913,73.908312,69.860500,61.872122,53.912896,53.210025,42.640912,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4551,2023-01-02T00:00:00,149.361835,137.589241,114.928348,104.305588,93.451777,86.232441,75.646703,71.706137,63.506715,54.615775,54.615775,41.912019,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4552,2023-01-03T00:00:00,151.920674,142.021339,118.325356,107.268621,96.462095,89.010206,78.090374,74.015973,65.552427,56.375087,56.375087,41.912019,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663
4553,2023-01-04T00:00:00,146.318325,138.068391,114.653918,104.036149,93.777220,86.532743,75.910886,71.955851,63.727875,54.805972,54.805972,41.912019,63.26991,52.113412,57.618272,50.858529,48.843833,45.716266,42.366663


In [6]:
# Export the data to a .csv file
df.to_csv(
    './datasets/cbeci-annualized-consumption.csv', 
    index = False
)